# 关键词提取算法

## 关键词提取技术概述

- 有监督的关键词提取方法
- 无监督的关键词提取方法：TF-IDF算法、TextRank算法和主题模型算法（包括LSA、LSI、LDA等）

## 关键词提取算法TF-IDF算法

<b>TF-IDF算法由两部分组成：TF算法和IDF算法。</b>   
- <b>TF算法：</b>统计一个词在一篇文档中出现的频次。基本思想是：一个词在文档中出现的次数越多，则其对文档的表达能力也就越强。   
- <b>IDF算法：</b>统计一个词在文档集的多少个文档中出现。基本思想是：如果一个词在越少的文档中出现，则其对文档的区分能力也就越强。

## TextRank算法

TextRank算法最早用于文档的自动摘要，基于句子维度的分析，利用TextRank对每个句子进行打分，挑选出分数最高的n个句子作为文档的关键词，以达到自动摘要的效果。

TextRank算法的基本思想来源于PageRank算法。   
PageRank算法是一种网页排名算法，基本思想有两条：   
1. <b>链接数量：</b>一个网页被越多的其他网页链接，说明这个网页越重要。
2. <b>链接质量：</b>一个网页被一个越高权值的网页链接，也能表明这个网页越重要。

## LSA/LSI/LDA算法

### LSA/LSI算法

LSA（潜在语义分析）和LSI（潜在语义索引），二者通常被认为是同一种算法，只是应用的场景略有不同。

LSA的主要步骤如下：     
1. 使用BOW模型将每个文档表示为向量
2. 将所有的文档词向量拼接起来构成词-文档矩阵（mxn)
3. 对词-文档矩阵进行奇异值分解（SVD）操作（[mxr][rxr][kxn]）
4. 根据SVD的结果，将词-文档矩阵映射到一个更低纬度k（[mxk][kxk][kxn],0<k<r）的近似SVD结果，每个词和文档都可以表示为k个主题构成的空间中的一个点，通过计算每个词和文档的相似度（余弦相似度或KL相似度），可以得到每个文档中对每个词的相似度结果，取相似度最高的一个词即为文档的关键词。

### LDA算法

LDA（隐含狄利克雷分布）算法

## 实战提取文本关键词

训练一个关键词提取算法需要以下几个步骤：  
1. 加载已有的文档数据集
2. 加载停用词表
3. 对数据集中的文档进行分词
4. 根据停用词表，过滤干扰词
5. 根据数据集训练算法

根据训练好的关键词提取算法对新文档进行关键词提取要经过以下环节：
1. 对新文档进行分词
2. 根据停用词表，过滤干扰词
3. 根据训练好的算法提取关键词

In [1]:
# 导入用到的外部库
import math
import jieba
import jieba.posseg as psg
from gensim import corpora, models
from jieba import analyse
import functools

In [2]:
# 停用词表加载方法
def get_stopword_list():
    # 停用词表存储路径，每一行为一个词，按行读取进行加载
    # 进行编码转换确保匹配准确率
    stop_word_path = './stopword.txt'
    stopword_list = [sw.replace('\n', '') for sw in open(stop_word_path).readlines()]
    return stopword_list

In [3]:
# 分词方法，调用结巴接口
def seg_to_list(sentence, pos=False):
    if not pos:
        # 不进行词性标注的分词方法
        seg_list = jieba.cut(sentence)
    else:
        # 进行词性标注的分词方法
        seg_list = psg.cut(sentence)
    return seg_list

In [4]:
# 去除干扰词
def word_filter(seg_list, pos=False):
    stopword_list = get_stopword_list()
    filter_list = []
    # 根据POS参数选择是否词性过滤
    ## 不进行词性过滤，则将词性都标记为n，表示全部保留
    for seg in seg_list:
        if not pos:
            word = seg
            flag = 'n'
        else:
            word = seg.word
            flag = seg.flag
        if not flag.startswith('n'):
            continue
        # 过滤停用词表中的词，以及长度为<2的词
        if not word in stopword_list and len(word) > 1:
            filter_list.append(word)

    return filter_list

In [5]:
# 数据加载，pos为是否词性标注的参数，corpus_path为数据集路径
def load_data(pos=False, corpus_path='./corpus.txt'):
    # 调用上面方式对数据集进行处理，处理后的每条数据仅保留非干扰词
    doc_list = []
    for line in open(corpus_path, 'r'):
        content = line.strip()
        seg_list = seg_to_list(content, pos)
        filter_list = word_filter(seg_list, pos)
        doc_list.append(filter_list)

    return doc_list

In [6]:
# idf值统计方法
def train_idf(doc_list):
    idf_dic = {}
    # 总文档数
    tt_count = len(doc_list)

    # 每个词出现的文档数
    for doc in doc_list:
        for word in set(doc):
            idf_dic[word] = idf_dic.get(word, 0.0) + 1.0

    # 按公式转换为idf值，分母加1进行平滑处理
    for k, v in idf_dic.items():
        idf_dic[k] = math.log(tt_count / (1.0 + v))

    # 对于没有在字典中的词，默认其仅在一个文档出现，得到默认idf值
    default_idf = math.log(tt_count / (1.0))
    return idf_dic, default_idf

In [7]:
#  排序函数，用于topK关键词的按值排序
def cmp(e1, e2):
    import numpy as np
    res = np.sign(e1[1] - e2[1])
    if res != 0:
        return res
    else:
        a = e1[0] + e2[0]
        b = e2[0] + e1[0]
        if a > b:
            return 1
        elif a == b:
            return 0
        else:
            return -1

In [8]:
# TF-IDF类
class TfIdf(object):
    # 四个参数分别是：训练好的idf字典，默认idf值，处理后的待提取文本，关键词数量
    def __init__(self, idf_dic, default_idf, word_list, keyword_num):
        self.word_list = word_list
        self.idf_dic, self.default_idf = idf_dic, default_idf
        self.tf_dic = self.get_tf_dic()
        self.keyword_num = keyword_num

    # 统计tf值
    def get_tf_dic(self):
        tf_dic = {}
        for word in self.word_list:
            tf_dic[word] = tf_dic.get(word, 0.0) + 1.0

        tt_count = len(self.word_list)
        for k, v in tf_dic.items():
            tf_dic[k] = float(v) / tt_count

        return tf_dic

    # 按公式计算tf-idf
    def get_tfidf(self):
        tfidf_dic = {}
        for word in self.word_list:
            idf = self.idf_dic.get(word, self.default_idf)
            tf = self.tf_dic.get(word, 0)

            tfidf = tf * idf
            tfidf_dic[word] = tfidf

        tfidf_dic.items()
        # 根据tf-idf排序，去排名前keyword_num的词作为关键词
        for k, v in sorted(tfidf_dic.items(), key=functools.cmp_to_key(cmp), reverse=True)[:self.keyword_num]:
            print(k + "/ ", end='')
        print()

In [9]:
# 主题模型
class TopicModel(object):
    # 三个传入参数：处理后的数据集，关键词数量，具体模型（LSI、LDA），主题数量
    def __init__(self, doc_list, keyword_num, model='LSI', num_topics=4):
        # 使用gensim的接口，将文本转为向量化表示
        # 先构建词空间
        self.dictionary = corpora.Dictionary(doc_list)
        # 使用BOW模型向量化
        corpus = [self.dictionary.doc2bow(doc) for doc in doc_list]
        # 对每个词，根据tf-idf进行加权，得到加权后的向量表示
        self.tfidf_model = models.TfidfModel(corpus)
        self.corpus_tfidf = self.tfidf_model[corpus]

        self.keyword_num = keyword_num
        self.num_topics = num_topics
        # 选择加载的模型
        if model == 'LSI':
            self.model = self.train_lsi()
        else:
            self.model = self.train_lda()

        # 得到数据集的主题-词分布
        word_dic = self.word_dictionary(doc_list)
        self.wordtopic_dic = self.get_wordtopic(word_dic)

    def train_lsi(self):
        lsi = models.LsiModel(self.corpus_tfidf, id2word=self.dictionary, num_topics=self.num_topics)
        return lsi

    def train_lda(self):
        lda = models.LdaModel(self.corpus_tfidf, id2word=self.dictionary, num_topics=self.num_topics)
        return lda

    def get_wordtopic(self, word_dic):
        wordtopic_dic = {}

        for word in word_dic:
            single_list = [word]
            wordcorpus = self.tfidf_model[self.dictionary.doc2bow(single_list)]
            wordtopic = self.model[wordcorpus]
            wordtopic_dic[word] = wordtopic
        return wordtopic_dic

    # 计算词的分布和文档的分布的相似度，取相似度最高的keyword_num个词作为关键词
    def get_simword(self, word_list):
        sentcorpus = self.tfidf_model[self.dictionary.doc2bow(word_list)]
        senttopic = self.model[sentcorpus]

        # 余弦相似度计算
        def calsim(l1, l2):
            a, b, c = 0.0, 0.0, 0.0
            for t1, t2 in zip(l1, l2):
                x1 = t1[1]
                x2 = t2[1]
                a += x1 * x1
                b += x1 * x1
                c += x2 * x2
            sim = a / math.sqrt(b * c) if not (b * c) == 0.0 else 0.0
            return sim

        # 计算输入文本和每个词的主题分布相似度
        sim_dic = {}
        for k, v in self.wordtopic_dic.items():
            if k not in word_list:
                continue
            sim = calsim(v, senttopic)
            sim_dic[k] = sim

        for k, v in sorted(sim_dic.items(), key=functools.cmp_to_key(cmp), reverse=True)[:self.keyword_num]:
            print(k + "/ ", end='')
        print()

    # 词空间构建方法和向量化方法，在没有gensim接口时的一般处理方法
    def word_dictionary(self, doc_list):
        dictionary = []
        for doc in doc_list:
            dictionary.extend(doc)

        dictionary = list(set(dictionary))

        return dictionary

    def doc2bowvec(self, word_list):
        vec_list = [1 if word in word_list else 0 for word in self.dictionary]
        return vec_list

In [10]:
# TF-IDF算法封装
def tfidf_extract(word_list, pos=False, keyword_num=10):
    doc_list = load_data(pos)
    idf_dic, default_idf = train_idf(doc_list)
    tfidf_model = TfIdf(idf_dic, default_idf, word_list, keyword_num)
    tfidf_model.get_tfidf()

In [11]:
# TextRank算法封装
def textrank_extract(text, pos=False, keyword_num=10):
    textrank = analyse.textrank
    keywords = textrank(text, keyword_num)
    # 输出抽取出的关键词
    for keyword in keywords:
        print(keyword + "/ ", end='')
    print()

In [12]:
# 主题模型LSI、LDA算法封装
def topic_extract(word_list, model, pos=False, keyword_num=10):
    doc_list = load_data(pos)
    topic_model = TopicModel(doc_list, keyword_num, model=model)
    topic_model.get_simword(word_list)

In [13]:
# 主函数封装
def main():
    text = '6月19日,《2012年度“中国爱心城市”公益活动新闻发布会》在京举行。' + \
           '中华社会救助基金会理事长许嘉璐到会讲话。基金会高级顾问朱发忠,全国老龄' + \
           '办副主任朱勇,民政部社会救助司助理巡视员周萍,中华社会救助基金会副理事长耿志远,' + \
           '重庆市民政局巡视员谭明政。晋江市人大常委会主任陈健倩,以及10余个省、市、自治区民政局' + \
           '领导及四十多家媒体参加了发布会。中华社会救助基金会秘书长时正新介绍本年度“中国爱心城' + \
           '市”公益活动将以“爱心城市宣传、孤老关爱救助项目及第二届中国爱心城市大会”为主要内容,重庆市' + \
           '、呼和浩特市、长沙市、太原市、蚌埠市、南昌市、汕头市、沧州市、晋江市及遵化市将会积极参加' + \
           '这一公益活动。中国雅虎副总编张银生和凤凰网城市频道总监赵耀分别以各自媒体优势介绍了活动' + \
           '的宣传方案。会上,中华社会救助基金会与“第二届中国爱心城市大会”承办方晋江市签约,许嘉璐理' + \
           '事长接受晋江市参与“百万孤老关爱行动”向国家重点扶贫地区捐赠的价值400万元的款物。晋江市人大' + \
           '常委会主任陈健倩介绍了大会的筹备情况。'

    pos = True
    seg_list = seg_to_list(text, pos)
    filter_list = word_filter(seg_list, pos)

    print('TF-IDF模型结果：')
    tfidf_extract(filter_list)
    print('TextRank模型结果：')
    textrank_extract(text)
    print('LSI模型结果：')
    topic_extract(filter_list, 'LSI', pos)
    print('LDA模型结果：')
    topic_extract(filter_list, 'LDA', pos)

In [14]:
main()

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 2.138 seconds.
Prefix dict has been built succesfully.


TF-IDF模型结果：
晋江市/ 城市/ 大会/ 爱心/ 中华/ 基金会/ 陈健倩/ 重庆市/ 许嘉璐/ 巡视员/ 
TextRank模型结果：
城市/ 爱心/ 救助/ 中国/ 社会/ 晋江市/ 基金会/ 大会/ 介绍/ 公益活动/ 
LSI模型结果：
中国/ 中华/ 爱心/ 基金会/ 项目/ 社会/ 城市/ 公益活动/ 全国/ 国家/ 
LDA模型结果：
晋江市/ 年度/ 大会/ 副理事长/ 爱心/ 理事长/ 优势/ 陈健倩/ 重庆市/ 巡视员/ 


In [16]:
doc_list = load_data(True)

In [18]:
dictionary = corpora.Dictionary(doc_list)

In [19]:
corpus = [dictionary.doc2bow(doc) for doc in doc_list]

In [21]:
tfidf_model = models.TfidfModel(corpus)

In [25]:
print(tfidf_model[corpus[1]])

[(58, 0.2134815890528836), (59, 0.1463231642699445), (60, 0.24706080144435316), (61, 0.2134815890528836), (62, 0.24706080144435316), (63, 0.24706080144435316), (64, 0.24706080144435316), (65, 0.19383900900012271), (66, 0.24706080144435316), (67, 0.24706080144435316), (68, 0.24706080144435316), (69, 0.24706080144435316), (70, 0.1602597966086532), (71, 0.24706080144435316), (72, 0.24706080144435316), (73, 0.10441874124686389), (74, 0.14061721655589232), (75, 0.24706080144435316), (76, 0.24706080144435316), (77, 0.24706080144435316)]


In [26]:
dictionary = []
for doc in doc_list:
    dictionary.extend(doc)
dictionary = list(set(dictionary))
print(dictionary)

['出厂', '临时代办', '湖南人', '老崔', '报警', '波及', '监管部门', '急需', '邓飞', '梅溪村', '松乡纳沟', '师傅', '垃圾箱', '贡献力量', '反响强烈', '红十字会', '民政', '学生', '湖南', '研究所', '方来英', '监控点', '风尚', '体操队', '中国科学院', '飞镖', '兄弟', '司法鉴定', '毛巾', '新快报', '民间组织', '窗角', '城乡居民', '深圳市政府', '门卫', '宣传教育', '赣州', '环境', '短片', '技术', '作秀', '买房', '刺骨', '报告会', '南昌', '苏伟', '黄河', '短期内', '黄金', '户口', '民警', '长假', '桑叶', '小康', '鸡蛋', '水塔', '生物', '英语课', '案开庭', '名誉', '省城', '专门', '国家', '市面上', '总费用', '离境', '小摊', '住宿', '白领', '人类', '排水管道', '移民', '创业园', '孤儿', '门槛', '名称', '不法', '每公斤', '绿地率', '常熟市', '废弃物', '爱心', '变味', '庄先', '抚慰金', '采集器', '度日', '菜农', '摩洛哥', '反面教材', '汽车玻璃', '公寓', '北京奥运', '余楚媛', '重点部位', '危房', '慈善事业', '面罩', '山东', '中国银行', '日内瓦', '罗琼', '美容', '凳子', '全校学生', '愿景', '创办人', '员工', '钢筋', '程度', '类产品', '行列', '电镀', '风雨', '垃圾', '采供血', '室间隔', '图形', '杨团', '传统', '校长', '叔叔', '热线', '热身', '大字', '计生委', '司机', '窗户', '玉米', '长辈', '疙瘩', '坦言', '塔山', '志愿者', '茶叶', '双杠', '复地', '小铁锤', '层面', '志愿', '外交部', '素质', '公交车', '流入量', '儿女', '金三银', '暴力', '圣品', '香味', '自主权', '李冰冰', '在京举行', '买单', 